# Virtual env

In [1]:
import aiosmtpd
from smtplib import SMTP as Client
from pyotp import random_base32, HOTP

In [2]:
import sys
if sys.base_prefix==sys.prefix:
    raise RuntimeError("Ne tourne pas sur l'environnement virtuel")

# Création d'un handler
Test

In [3]:
import asyncio
class ExampleHandler:
    async def handle_RCPT(self, server, session, envelope, address, rcpt_options):
        try:
            extensions = (address.split("@", 1)[0]).split("+")
            hotp = int(extensions[1])
            print("HOTP:{%d}".format(hotp))
        except:
            return '550 not relaying to that domain'
        return '250 OK'

    async def handle_DATA(self, server, session, envelope):
        print('Message from %s' % envelope.mail_from)
        print('Message options %s' % str(envelope.mail_options))
        print('Message for %s' % envelope.rcpt_tos)
        print('Message data:\n')
        for ln in envelope.content.decode('utf8', errors='replace').splitlines():
            print(f'> {ln}'.strip())
        print()
        print('End of message')
        return '250 Message accepted for delivery'

In [4]:
from aiosmtpd.controller import Controller
controller = Controller(ExampleHandler())
controller.start()

## Test du handler

In [5]:
client = Client(controller.hostname, controller.port)
r = client.sendmail('a+123456@example.com', ['b@example.com'], """\
From: Alice <alice@example.com>
To: Bob <bob@example.com>
Subject: Test co
Message-ID: <ant>

Hello.""")

SMTPRecipientsRefused: {'b@example.com': (550, b'not relaying to that domain')}

In [6]:
client = Client(controller.hostname, controller.port)
r = client.sendmail('a@example.com', ['b@pasexample.com'], """\
From: Alice <alice@example.com>
To: Bob <bob@example.com>
Subject: Test co
Message-ID: <ant>

Hello.""")

SMTPRecipientsRefused: {'b@pasexample.com': (550, b'not relaying to that domain')}

In [66]:
client = Client(controller.hostname, controller.port)
r = client.sendmail('a+test@example.com', ['b@example.com'], """\
From: Alice <alice@example.com>
To: Bob <bob@example.com>
Subject: Test co
Message-ID: <ant>

Hello.""")

SMTPRecipientsRefused: {'b@example.com': (550, b'not relaying to that domain')}

# Extinction du serveur SMTP
et contrôle de l'extinction

In [67]:
controller.stop()

try:
    client.connect(controller.hostname, controller.port)
    print("!!! Le serveur n'est pas éteint")
except ConnectionRefusedError:
    print("Serveur éteint")
except Exception as err:
    print(f"Unexpected {err=}, {type(err)=}")

Serveur éteint


# HOTP génération et utilisation

In [54]:
secretKey = random_base32()
hotp = HOTP(secretKey)
print(secretKey)

3B7KHDWFFZ5LXPEEKQQEF7TB56GUPQHQ


In [55]:
print(hotp.at(2),
    hotp.at(5),
    hotp.at(10))


548211 433270 607343


In [57]:
rcptHotp = HOTP(secretKey)
rcptHotp.verify("548211", 2)

True

In [58]:
len("123456")

6